In [ ]:
!pip install catboost

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, roc_auc_score
import sklearn.metrics as metrics

from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder

from catboost import CatBoostClassifier

In [33]:
train = pd.read_csv('/content/train_dataset.csv', low_memory=False)

In [4]:
def remove_repetitions(first_list, second_list):
    return [element for element in first_list if element not in second_list]

=================================================== ОБРАБОТКА КАТЕГОРИАЛЬНЫХ ДАННЫХ ===================================================

In [34]:
cat_cols = [col for col in train.columns if train[col].dtype == 'object']
cat_cols.pop(0)

train_cat = train[cat_cols]
train_cat = train_cat.fillna('')


train_cat_np = train_cat.to_numpy()

for i in range(len(train_cat_np)):
    for j in range(len(train_cat_np[i])):
        el = train_cat_np[i][j]
        if isinstance(el, int):
            continue

        if el != "":
            if "https" in el:
                new_i = el[el.find("//")+2:]
                new_i = new_i[:new_i.find("/")]
                train_cat_np[i][j] = new_i
train_cat = pd.DataFrame(train_cat_np, columns=cat_cols)

In [35]:
train_cat = train_cat.apply(LabelEncoder().fit_transform)

In [36]:
corr_matrix = train_cat.corr().abs()

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

train_cat.drop(to_drop, axis=1, inplace=True)

cat_cols = remove_repetitions(cat_cols, to_drop)

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ ОБРАБОТКА КАТЕГОРИАЛЬНЫХ ДАННЫХ ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

=================================================== ОБРАБОТКА ЧИСЛЕННЫХ ДАННЫХ ===================================================

In [37]:
num_cols = [col for col in train.columns if train[col].dtype != 'object']
num_cols.pop(1)
train_num = train[num_cols]
train_num = train_num.fillna(0)  # try mediana

In [38]:
corr_matrix = train_num.corr().abs()

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]

train_num.drop(to_drop, axis=1, inplace=True)

num_cols = remove_repetitions(num_cols, to_drop)

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ ОБРАБОТКА ЧИСЛЕННЫХ ДАННЫХ ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [39]:
# СОЕДИНЕНИЕ ОБРАБОТАННЫХ ДАННЫХ
train_norm = train_num.join(train_cat)
train_norm

,client_id,col1,col2,col4,col5,col7,col8,col45,col46,col47,...,col1647,col1649,col1650,col2191,col2192,col2193,col2194,col2195,col2196,col2197
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14451,1241,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
14452,1969,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
14453,7116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
14454,7117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
stand_sc = StandardScaler()

# data_ohe = pd.DataFrame(onehotencoder.fit_transform(train[cat_cols]))
data_num_cols = stand_sc.fit_transform(train_norm[num_cols])
data_norm = pd.DataFrame(data_num_cols, columns=num_cols).join(train_norm[cat_cols])

y = train['target'].values
X = data_norm[data_norm.columns[:-1]].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=62, stratify=y)
print(np.unique(y_train), np.unique(y_test))

[0 1] [0 1]


In [13]:
X_train.shape

(11564, 770)

In [ ]:
# XGBOOST EXECUTION

import xgboost
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

DMatTrain = xgboost.DMatrix(X_train, y_train)
DMatTest  = xgboost.DMatrix(X_test,  y_test )

booster = xgboost.train(params={'max_depth' : 10, "objective" : "binary:logistic" }, dtrain=DMatTrain, evals=[ (DMatTrain, "train"), (DMatTest, "test") ])

print("Train RMSE: ", booster.eval(DMatTrain))
print("Test RMSE:  ", booster.eval(DMatTest ))

train_preds = [1 if pred>0.5 else 0 for pred in booster.predict(data=DMatTrain)]
test_preds  = [1 if pred>0.5 else 0 for pred in booster.predict(data=DMatTest )]

print("\nTest  Accuracy : %.2f"%accuracy_score(y_test, test_preds))
print("Train Accuracy : %.2f"%accuracy_score(y_train, train_preds))

print("\nConfusion Matrix : ")
print(confusion_matrix(y_test, test_preds))

print("\nClassification Report : ")
print(classification_report(y_test, test_preds))

print("\nRoc Auc Score: ")
print(roc_auc_score(y_test, test_preds))

In [ ]:
import random
all_data: dict = []
for _ in range(100):
    n_estimators = random.randint(100, 600)
    max_depth = random.randint(10, 50)
    learning_rate = random.uniform(0.1, 0.9)

    xgb_classifier = xgboost.XGBClassifier(n_estimators = n_estimators, max_depth = max_depth, learning_rate = learning_rate)

    xgb_classifier.fit(X_train, y_train, eval_set = [ (X_test, y_test) ], eval_metric = "auc", verbose = 3)

    print("Test  Accuracy Score : %.2f"%xgb_classifier.score(X_test, y_test))
    print("Train Accuracy Score : %.2f"%xgb_classifier.score(X_train, y_train))

    print("\nRoc Auc Score: ")
    test_preds = xgb_classifier.predict(X_test)
    roc = roc_auc_score(y_test, test_preds)

    all_data[str([n_estimators, max_depth, learning_rate])] = roc

    print(roc)

print(all_data)

In [41]:
from catboost import Pool, EShapCalcType, EFeaturesSelectionAlgorithm

feature_names = ['F{}'.format(i) for i in range(X_train.shape[1])]
train_pool = Pool(X_train, y_train, feature_names=feature_names)
test_pool = Pool(X_test, y_test, feature_names=feature_names)

model = CatBoostClassifier(iterations=1000, random_seed=0, task_type="GPU")
summary = model.select_features(
    train_pool,
    eval_set=test_pool,
    features_for_select=feature_names,
    num_features_to_select=50,
    steps=3,
    algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues,
    shap_calc_type=EShapCalcType.Regular,
    train_final_model=True,
    logging_level='Verbose',
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Выходные данные были обрезаны до нескольких последних строк (5000).
224:	learn: 0.0811059	test: 0.1017509	best: 0.1017509 (224)	total: 11.3s	remaining: 38.9s
225:	learn: 0.0806733	test: 0.1016066	best: 0.1016066 (225)	total: 11.3s	remaining: 38.8s
226:	learn: 0.0806236	test: 0.1016575	best: 0.1016066 (225)	total: 11.4s	remaining: 38.7s
227:	learn: 0.0805788	test: 0.1016505	best: 0.1016066 (225)	total: 11.4s	remaining: 38.7s
228:	learn: 0.0805162	test: 0.1016913	best: 0.1016066 (225)	total: 11.5s	remaining: 38.7s
229:	learn: 0.0804686	test: 0.1017015	best: 0.1016066 (225)	total: 11.5s	remaining: 38.6s
230:	learn: 0.0804446	test: 0.1016923	best: 0.1016066 (225)	total: 11.6s	remaining: 38.5s
231:	learn: 0.0803135	test: 0.1016162	best: 0.1016066 (225)	total: 11.6s	remaining: 38.5s
232:	learn: 0.0803091	test: 0.1016161	best: 0.1016066 (225)	total: 11.7s	remaining: 38.4s
233:	learn: 0.0802367	test: 0.1015574	best: 0.1015574 (233)	total: 11.7s	remaining: 38.3s
234:	learn: 0.0801818	test: 0.10

In [42]:
feature_importance = model.get_feature_importance(train_pool)
feature_importance
feature_importance.shape[0]

print(len(feature_names))
type(feature_importance)
print(feature_importance.shape[0])
indexes = np.where(feature_importance > 0)

# Выведите индексы элементов больше 0
print(indexes)

result = feature_importance[feature_importance > 0]

print(result)
type(result)

result.shape[0]


1260
1260
(array([   0,  143,  169,  289,  394,  395,  584,  704,  722,  782,  784,
        787,  788,  789,  804,  831,  832,  834,  837,  843,  846,  847,
        848,  849,  857,  859,  860,  861,  862,  865,  866,  867,  868,
        876,  882,  883,  884,  886,  887,  893,  895,  956,  960,  972,
        978,  980,  981, 1090, 1234, 1250]),)
[7.9239179  0.67769852 0.29894818 0.19301966 1.23994984 4.10778629
 0.15585332 0.72378712 0.66893771 2.62045966 2.62412722 3.80195929
 1.35348715 3.49956724 1.63959463 0.28908953 0.27732557 1.99107685
 3.934261   0.75702416 2.1363762  1.74984042 4.01491959 3.64488274
 4.66912147 0.84997454 0.51015802 0.89826065 2.80664992 0.3899543
 1.72083916 0.19056896 2.17806403 2.27758045 0.79876938 1.05944051
 1.7033582  1.54507664 2.43983856 2.13199792 2.69863367 0.97269597
 1.56405942 1.25632154 1.08897977 1.23228151 9.70700642 0.38118357
 0.16119242 4.44410321]


50

In [43]:
type(data_norm)
columns_to_keep = [   0,  143,  169,  289,  394,  395,  584,  704,  722,  782,  784,
        787,  788,  789,  804,  831,  832,  834,  837,  843,  846,  847,
        848,  849,  857,  859,  860,  861,  862,  865,  866,  867,  868,
        876,  882,  883,  884,  886,  887,  893,  895,  956,  960,  972,
        978,  980,  981, 1090, 1234, 1250]

print(data_norm.shape)
print(len(columns_to_keep))


data_norm_new = data_norm.iloc[:, columns_to_keep]

print(data_norm_new.shape)

(14456, 1261)
50
(14456, 50)


In [44]:
# stand_sc = StandardScaler()

# # data_ohe = pd.DataFrame(onehotencoder.fit_transform(train[cat_cols]))
# data_num_cols = stand_sc.fit_transform(train_norm[num_cols])
# data_norm = pd.DataFrame(data_num_cols, columns=num_cols).join(train_norm[cat_cols])

y = train['target'].values
X = data_norm_new[data_norm_new.columns[:-1]].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=62, stratify=y)
print(np.unique(y_train), np.unique(y_test))

[0 1] [0 1]


In [ ]:
import xgboost

xgb_classifier = xgboost.XGBClassifier(params = {'updater' : 'refresh'}, n_estimators = 400, max_depth = 20, learning_rate = 0.1, booster = "dart", tree_method = "hist", objective = "reg:squaredlogerror",eval_metric = "auc") #300, 20, 0.4 - ideal params || boosters "gbtree" "dart"

# objective reg:squarederror exact tree method, booster gbtree 350, 20, 0.1

xgb_classifier.fit(X_train, y_train, eval_set = [ (X_test, y_test) ])

print("Test  Accuracy Score : %.2f"%xgb_classifier.score(X_test, y_test))
print("Train Accuracy Score : %.2f"%xgb_classifier.score(X_train, y_train))

print("\nRoc Auc Score: ")
test_preds = xgb_classifier.predict(X_test)
print(roc_auc_score(y_test, test_preds))

[00:57:13] WARNING: ../src/learner.cc:767: 
Parameters: { "params" } are not used.

[0]	validation_0-auc:0.79529
[1]	validation_0-auc:0.82431
[2]	validation_0-auc:0.85940
[3]	validation_0-auc:0.87864
[4]	validation_0-auc:0.87666
[5]	validation_0-auc:0.88661
[6]	validation_0-auc:0.88645
[7]	validation_0-auc:0.89333
[8]	validation_0-auc:0.88883
[9]	validation_0-auc:0.89351
[10]	validation_0-auc:0.92608
[11]	validation_0-auc:0.93198
[12]	validation_0-auc:0.93294
[13]	validation_0-auc:0.92881
[14]	validation_0-auc:0.92738
[15]	validation_0-auc:0.92869
[16]	validation_0-auc:0.93004
[17]	validation_0-auc:0.93256
[18]	validation_0-auc:0.93243
[19]	validation_0-auc:0.93452
[20]	validation_0-auc:0.93460
[21]	validation_0-auc:0.93354
[22]	validation_0-auc:0.93703
[23]	validation_0-auc:0.93680
[24]	validation_0-auc:0.93707
[25]	validation_0-auc:0.93690
[26]	validation_0-auc:0.93758
[27]	validation_0-auc:0.93803
[28]	validation_0-auc:0.93556
[29]	validation_0-auc:0.93725
[30]	validation_0-auc:0.94

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import time
start_time = time.time()

# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=2000, max_depth=100, verbose=2, min_samples_leaf=1,n_jobs=-1,bootstrap=False,criterion="log_loss")

# Fit the Random Forest classifier to the training data
rf_classifier.fit(X_train, y_train)


# Evaluate the model
print("Test  Accuracy Score : %.2f" % rf_classifier.score(X_test, y_test))
print("Train Accuracy Score : %.2f" % rf_classifier.score(X_train, y_train))

print("\nRoc Auc Score: ")
test_preds = rf_classifier.predict(X_test)
print(roc_auc_score(y_test, test_preds))
end_time = time.time()
execution_time = end_time - start_time
print(f"RandomForestClassifier took {execution_time:.2f} seconds")